In [1]:
import numpy as np
import functools as ft
import networkx as nx
from sklearn.base import BaseEstimator, ClassifierMixin
from datasets import load_dataset
from sklearn.model_selection import cross_val_score, ShuffleSplit

from graph import processDataset, transform
from hdc import hdv, bind, bundle, ItemMemory, hdvW, hdvA

/home/p4ssenger/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def encodeGraph(graph, vertices, dimensions):
    for node in graph.nodes:
        if node not in vertices:
            vertices[node] = hdv(dimensions)

    Edges = []

    for edge in graph.edges:
        v1 = vertices[edge[0]]
        v2 = vertices[edge[1]]
        E = bind([v1, v2])
        Edges.append(E)

    Graph = bundle(Edges)

    return Graph

In [3]:
class GraphHD(BaseEstimator, ClassifierMixin):
    def __init__(self, alpha=0.45, digits=4, dimensions=10000, step=20):
        self.alpha = alpha
        self.digits = digits
        self.dimensions = dimensions
        self.step = step

        self.memory = ItemMemory([])

    def fit(self, X, y):
        self.vertices = dict()
        self.labels = list(set(y))
        dictLabels = dict()

        graphs = transform(X, self.alpha, self.digits)

        for label in self.labels:
            dictLabels[label] = []

        for i in range(len(graphs)):
            Graph = encodeGraph(graphs[i], self.vertices, self.dimensions)
            dictLabels[y[i]].append(Graph)

        for key, value in dictLabels.items():
            for i in range(0, len(value), self.step):
                H = bundle(value[i : i + self.step])
                self.memory.addVector(str(key), H)

        return self

    def predict(self, X):
        p = []
        s = []

        graphs = transform(X, self.alpha, self.digits)

        for testGraph in graphs:
            queryVector = encodeGraph(testGraph, self.vertices, self.dimensions)
            cleanVector = self.memory.cleanup(queryVector)

            p.append(int(cleanVector[0]))
            # s.append(cosine_similarity(queryVector, cleanVector[1]))

        # print("%.5f" % round(np.mean(s), 5), "0:", p.count(0), "1:", p.count(1))
        return p

In [4]:
MUTAG = load_dataset("graphs-datasets/MUTAG")["train"]
PROTEINS = load_dataset("graphs-datasets/PROTEINS")["train"]
IMDB = load_dataset("graphs-datasets/IMDB-BINARY")["train"]

In [5]:
(graphs, labels) = processDataset(MUTAG)

In [10]:
FOLDS, REPS = 10, 10
ALPHA, DIGITS, DIMENSIONS, STEP = 0.85, 8, 10000, 50
CV = FOLDS


def main():
    clf = GraphHD(ALPHA, DIGITS, DIMENSIONS, STEP)
    sum = 0
    for i in range(REPS):
        # CV = ShuffleSplit(n_splits=5, test_size=0.25)  # random_state=0
        scores = cross_val_score(clf, graphs, labels, n_jobs=-1, cv=CV, verbose=0)
        sum += scores.mean()
        print(i, "->", "%.5f" % scores.mean())
        del scores

    print("S => %.5f" % (sum / REPS))


main()

0 -> 0.67865
1 -> 0.71228
2 -> 0.65263
3 -> 0.72310
4 -> 0.65322
5 -> 0.68070
6 -> 0.68480
7 -> 0.65789
8 -> 0.65848
9 -> 0.65263
S => 0.67544
